## CausalANOVA Implementation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import class_FM as FM
import rpy2.robjects as ro

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from sklearn import preprocessing

In [3]:
# Import required R packages
utils = importr('utils')
utils.chooseCRANmirror(ind=66)
utils.install_packages('FindIt')
base = importr("base")
findit = importr("FindIt")

R[write to console]: trying URL 'https://cran.uib.no/src/contrib/FindIt_1.2.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 150340 bytes (146 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

### Initialize factorial model and sample dataset

In [4]:
n = 1000
k = 3
degree = 3
sigma = 0.1
sparsity = 0.5

fm = FM.FactorialModel(
    n=n,
    k=k,
    degree=degree,
    sigma=sigma,
    sparsity=sparsity,
    beta_seed=42,
)
t, y = fm.sample(seed=0)

### Fit OLS model with sample data

In [5]:
T = preprocessing.PolynomialFeatures(
    degree=degree, interaction_only=True, include_bias=True,
).fit_transform(t)
m = sm.OLS(y, T)
results = m.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 2.093e+04
Date:                Tue, 19 Mar 2024   Prob (F-statistic):               0.00
Time:                        21:54:13   Log-Likelihood:                 881.00
No. Observations:                1000   AIC:                            -1746.
Df Residuals:                     992   BIC:                            -1707.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0085      0.009      0.964      0.3

### Fit CausalANOVA model with sample dataset

In [6]:
# Convert NumPy arrays to pandas DataFrame
data = pd.DataFrame(t.astype("str"), columns=["t1", "t2", "t3"], dtype="category")
data["outcome"] = y

# Convert pandas DataFrame to R data.frame
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)

ro.globalenv["r_data"] = r_data

fac_level = ro.IntVector([2, 2, 2])
ro.globalenv["fac_level"] = fac_level

_ = ro.r(
    """
fit <- CausalANOVA(formula=outcome ~ t1 + t2 + t3, family="gaussian", data=r_data, nway=3)
"""
)
print(ro.r("summary(fit)"))


Check: the number of levels for factors and whether they are ordered.
   levels ordered
t1      2   FALSE
t2      2   FALSE
t3      2   FALSE

Analyzing all two-way and three-way interactions...
 
Model:
 outcome ~ . * . * .
<environment: 0x17290d7e8>
******
Range:
******
         range
t1       2.095
t2       0.991
t3       0.019
t1:t2    1.046
t1:t3    0.082
t2:t3    0.019
t1:t2:t3 0.078
*******************************
Average Marginal Effects (AME):
*******************************
 Factor Levels base    AME Std.Err 2.5%CI 97.5%CI
     t1    0.0      2.0954  0.0064 2.0828  2.1080
     t1    1.0  *** 0.0000  0.0000 0.0000  0.0000
     t2    0.0      0.9912  0.0064 0.9786  1.0037
     t2    1.0  *** 0.0000  0.0000 0.0000  0.0000
     t3    0.0      0.0188  0.0064 0.0062  0.0314
     t3    1.0  *** 0.0000  0.0000 0.0000  0.0000

********************************************
Average Marginal Interaction Effects (AMIE):
********************************************
Two-way Interactions:
 F